## Data cleaning
### This notebook is for cleaning the house pricing data

Importing packages

In [3]:
try:
    import pandas as pd
    import plotly.express as px 
    import numpy as np
except:
    !pip install pandas
    !pip install plotly
    import pandas as pd
    import plotly.express as px 

Loading the data

In [4]:
corr_data = pd.read_pickle('../data/house_pricing_corr.pkl')

First I inspect the empty cells column wise